In [1]:
# import the necessary packages
import cv2
class ShapeDetector:
    def __init__(self):
        pass
    def detect(self, c):
        # initialize the shape name and approximate the contour
        shape = "unidentified"
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        # if the shape is a triangle, it will have 3 vertices
        if len(approx) == 3:
            shape = "triangle"
        # if the shape has 4 vertices, it is either a square or
        # a rectangle
        elif len(approx) == 4:
            # compute the bounding box of the contour and use the
            # bounding box to compute the aspect ratio
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h)
            # a square will have an aspect ratio that is approximately
            # equal to one, otherwise, the shape is a rectangle
            shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"
        # if the shape is a pentagon, it will have 5 vertices
        elif len(approx) == 5:
            shape = "pentagon"
        elif len(approx) == 6:
            shape = "hexagon"
        # otherwise, we assume the shape is a circle
        else:
            shape = "circle"
        # return the name of the shape
        return shape

In [2]:
import cv2
import os
import numpy as np
def process_image(image_path, show_result = False):
    # reading image
    img = cv2.imread(image_path)
    image_name = os.path.basename(image_path)

    # converting image into grayscale image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite("result\gray_" + image_name, gray)
    
    # setting threshold of gray image
    h, w = gray.shape[:2]
    m = np.reshape(gray, [1, w*h])
    mean = m.sum()/(w*h)
    ret, threshold = cv2.threshold(gray, mean, 255, cv2.THRESH_BINARY)
    
    #threshold = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 25, 10)
    #threshold = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
    cv2.imwrite(r"result\threshold_" + image_name, threshold)

    # using a findContours() function
    contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    i = 0
    sd = ShapeDetector()
    
    # list for storing names of shapes
    for contour in contours:
        # here we are ignoring first counter because
        # findcontour function detects whole image as shape
        if i == 0:
            i = 1
            continue
        # cv2.approxPloyDP() function to approximate the shape
        approx = cv2.approxPolyDP(contour, 0.01 * cv2.arcLength(contour, True), True)
        # using drawContours() function
        cv2.drawContours(img, [contour], 0, (0, 0, 255), 2)

        # finding center point of shape
        M = cv2.moments(contour)
        if M['m00'] != 0.0:
            x = int(M['m10']/M['m00'])
            y = int(M['m01']/M['m00'])
        
        shape = sd.detect(contour)
        
        
        cv2.putText(img, shape, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.2, (255, 255, 255), 1)
    
    cv2.imwrite("result\processed_" + image_path, img)
    # displaying the image after drawing contours
    if show_result:
        cv2.imshow('shapes', img)

        cv2.waitKey(0)
        cv2.destroyAllWindows()


In [20]:
process_image("tmp.png", True)

In [3]:
# UI part
import tkinter as tk
from tkinter import messagebox
from tkinter import * 
from tkinter import ttk
from tkinter import filedialog
from PIL import ImageTk,Image
import os

filename = ""
def UploadAction(event=None):
    global filename
    filename = filedialog.askopenfilename()
    e = entry_filename
    e.delete(0,END)
    e.insert(0,filename)
    print('Selected:', filename)
    UpdateImage(filename, lbl_image)
    
def UpdateImage(image_path, lbl_image):
    img = Image.open(image_path)
    print(img)
    ratio = img.size[1] / img.size[0]
    photo = ImageTk.PhotoImage(img.resize((400, int(400 * ratio))))
    lbl_image.configure(image=photo)
    lbl_image.image = photo
    return img

def ProcessImage(event=None):
    process_image(filename)
    image_name = os.path.basename(filename)
    UpdateImage("result\processed_" + image_name, lbl_processed_image)

# root window
root = tk.Tk()
root.geometry("1000x500")
root.title('Geometric shape detection')
#root.resizable(0, 0)

# configure the grid
root.columnconfigure(0, weight=3)
root.columnconfigure(1, weight=3)

# entry_filename
btn = tk.Button(root, text='Upload image', command=UploadAction, width=15)
btn.grid(column=0, row=0, sticky=tk.W, padx=5, pady=5)

entry_filename = ttk.Entry(root, width=66)
entry_filename.grid(column=0, row=1, sticky=tk.W, padx=5, pady=5)

image_path = "place.png"
img = Image.open(image_path)
ratio = img.size[0] / img.size[1]
photo = ImageTk.PhotoImage(img.resize((400, int(400 * ratio))))

lbl_image = Label(root, image=photo)
lbl_image.image = photo
lbl_image.grid(column=0, row=2, sticky=tk.W, padx=5, pady=5)

btn_process = tk.Button(root, text='Process image', command=ProcessImage, width=15)
btn_process.grid(column=1, row=0, sticky=tk.W, padx=5, pady=5)

lbl_processed_image = Label(root, image=photo)
lbl_processed_image.image = photo
lbl_processed_image.grid(column=1, row=2, sticky=tk.W, padx=5, pady=5)

root.mainloop()

In [ ]:
from PIL import ImageGrab

def save_image(widget, path):
    x=widget.winfo_x() + widget.winfo_rootx()
    y=widget.winfo_y() + widget.winfo_rooty() - 40
    x1=x+widget.winfo_width()
    y1=y+widget.winfo_height()
    ImageGrab.grab((x,y,x1,y1)).save(path)

In [26]:
from tkinter import *
from tkinter import ttk, colorchooser, filedialog
import PIL
from PIL import ImageGrab


class main:
    def __init__(self,master):
        self.master = master
        self.color_fg = 'black'
        self.color_bg = 'white'
        self.old_x = None
        self.old_y = None
        self.penwidth = 5
        self.drawWidgets()
        self.c.bind('<B1-Motion>',self.paint)
        self.c.bind('<ButtonRelease-1>',self.reset)

    def paint(self,e):
        if self.old_x and self.old_y:
            self.c.create_line(self.old_x,self.old_y,e.x,e.y,width=self.penwidth,fill=self.color_fg,capstyle=ROUND,smooth=True)

        self.old_x = e.x
        self.old_y = e.y

    def reset(self,e):
        self.old_x = None
        self.old_y = None      

    def changeW(self,e):
        self.penwidth = e

    def save(self):
        file = filedialog.asksaveasfilename(filetypes=[('Portable Network Graphics','*.png')])
        if file:
            x = self.master.winfo_rootx() + self.c.winfo_x()
            y = self.master.winfo_rooty() + self.c.winfo_y()
            x1 = x + self.c.winfo_width()
            y1 = y + self.c.winfo_height()

            PIL.ImageGrab.grab().crop((x,y,x1,y1)).save(file + '.png')
            
           

    def clear(self):
        self.c.delete(ALL)

    def hint(self):
        image_path = "tmp.png"
        save_image(self.c, image_path)
        process_image(image_path, show_result=True)
        
    def change_fg(self):
        self.color_fg=colorchooser.askcolor(color=self.color_fg)[1]

    def change_bg(self):
        self.color_bg=colorchooser.askcolor(color=self.color_bg)[1]
        self.c['bg'] = self.color_bg

    def drawWidgets(self):
        self.controls = Frame(self.master,padx = 5,pady = 5)
        Label(self.controls, text='Pen Width: ',font=('',15)).grid(row=0,column=0)
        self.slider = ttk.Scale(self.controls,from_= 5, to = 100, command=self.changeW,orient=HORIZONTAL)
        self.slider.set(self.penwidth)
        self.slider.grid(row=0,column=1,ipadx=30)
        self.controls.pack()
        
        self.c = Canvas(self.master,width=500,height=400,bg=self.color_bg,)
        self.c.pack(fill=BOTH,expand=True)

        menu = Menu(self.master)
        self.master.config(menu=menu)
        filemenu = Menu(menu)
        menu.add_cascade(label='File..',menu=filemenu)
        filemenu.add_command(label='Export..',command=self.save)
        colormenu = Menu(menu)
        menu.add_cascade(label='Colors',menu=colormenu)
        colormenu.add_command(label='Brush Color',command=self.change_fg)
        colormenu.add_command(label='Background Color',command=self.change_bg)
        optionmenu = Menu(menu)
        menu.add_cascade(label='Options',menu=optionmenu)
        optionmenu.add_command(label='Hints',command=self.hint)
        optionmenu.add_command(label='Clear Canvas',command=self.clear)
        optionmenu.add_command(label='Exit',command=self.master.destroy) 
        
        

if __name__ == '__main__':
    root = Tk()
    main(root)
    root.title('DrawingApp')
    root.mainloop()